In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn #module to implement neural network
import torch.nn.parallel #module for parallel computation
import torch.optim as optim #optimizer
import torch.utils.data  #stochastic gradient descent
from torch.autograd import Variable

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [3]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [6]:
#Architecture of Neural Network
class SAE(nn.Module):
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)
        

In [7]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
            
            


epoch: 1 loss: 1.77107419389
epoch: 2 loss: 1.09660767079
epoch: 3 loss: 1.05357298289
epoch: 4 loss: 1.03823606753
epoch: 5 loss: 1.03084868982
epoch: 6 loss: 1.02647616454
epoch: 7 loss: 1.02381205646
epoch: 8 loss: 1.02195534661
epoch: 9 loss: 1.02086297873
epoch: 10 loss: 1.01924520683
epoch: 11 loss: 1.01902831197
epoch: 12 loss: 1.01811693399
epoch: 13 loss: 1.0180942354
epoch: 14 loss: 1.01746349984
epoch: 15 loss: 1.01749741683
epoch: 16 loss: 1.01694427561
epoch: 17 loss: 1.01690387383
epoch: 18 loss: 1.01628427203
epoch: 19 loss: 1.01656385395
epoch: 20 loss: 1.01590900675
epoch: 21 loss: 1.01601061599
epoch: 22 loss: 1.01613011959
epoch: 23 loss: 1.01581187341
epoch: 24 loss: 1.01572600593
epoch: 25 loss: 1.01591434544
epoch: 26 loss: 1.01542817554
epoch: 27 loss: 1.01545810831
epoch: 28 loss: 1.01478207658
epoch: 29 loss: 1.01329703841
epoch: 30 loss: 1.01154436103
epoch: 31 loss: 1.00940871967
epoch: 32 loss: 1.00879545814
epoch: 33 loss: 1.00476118412
epoch: 34 loss: 1.00